In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/south-african-language-identification-2021/sample_submission.csv
/kaggle/input/south-african-language-identification-2021/test_set.csv
/kaggle/input/south-african-language-identification-2021/train_set.csv


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from datetime import datetime

# Load Data

## Import Libraries
Import the required libraries.

In [21]:
df_train = pd.read_csv("../input/south-african-language-identification-2021/train_set.csv")
df_test = pd.read_csv('../input/south-african-language-identification-2021/test_set.csv')

### Inspect DataFrames

In [22]:
df_train.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [23]:
df_test.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


In [24]:
df_test.shape

(5682, 2)

### Data Types

In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [26]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5682 entries, 0 to 5681
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   5682 non-null   int64 
 1   text    5682 non-null   object
dtypes: int64(1), object(1)
memory usage: 88.9+ KB


### Missing Data

In [27]:
# Identify missing data function.
def total_missing(df, column_name):
    miss_entr = df[column_name].isnull().sum()
    return miss_entr

In [28]:
total_missing(df_train, df_train.columns)

lang_id    0
text       0
dtype: int64

In [29]:
total_missing(df_test, df_test.columns)

index    0
text     0
dtype: int64

### Unique Data

In [30]:
df_train.nunique()

lang_id       11
text       29948
dtype: int64

In [31]:
df_train["lang_id"].value_counts()

sot    3000
xho    3000
eng    3000
ssw    3000
ven    3000
afr    3000
zul    3000
tso    3000
nbl    3000
tsn    3000
nso    3000
Name: lang_id, dtype: int64

### All Lower Case

In [32]:
df_train["text"] = df_train["text"].str.lower()

In [33]:
def to_lower(df):
    df["text"] = df["text"].str.lower()
    return df

In [34]:
df_train = to_lower(df_train)
df_test = to_lower(df_test)

## Feature Engineering

### CountVectorizer

In [35]:
def feat_CountVec(df):
    vect = CountVectorizer()
    vect.fit(df["text"])
    vector_cv = vect.transform(df["text"])
    return vector_cv

In [36]:
def feat_CountVec(train, test):
    vect = CountVectorizer()
    vect.fit(train)
    train_cv = vect.transform(train)
    test_cv = vect.transform(test)
    return train_cv, test_cv

In [37]:
train, test = feat_CountVec(df_train["text"], df_test["text"])

### TfidfVectorizer

In [38]:
def feat_TfidfVec(df):
    vect = TfidfVectorizer()
    vect.fit(df["text"])
    vector_tv = vect.transform(df["text"])
    return vector_tv

In [39]:
def feat_TfidfVec(train, test):
    vect = TfidfVectorizer()
    vect.fit(train)
    train_cv = vect.transform(train)
    test_cv = vect.transform(test)
    return train_cv, test_cv

In [40]:
train, test = feat_TfidfVec(df_train["text"], df_test["text"])

### Train Test Split

In [41]:
def train_split(df, X_feat):
    X = X_feat
    y = df["lang_id"]
    # split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return (X_train, y_train), (X_test, y_test)

In [42]:
(X_train, y_train), (X_test, y_test) = train_split(df_train, train)

# Modelling

## BernoulliNB

In [43]:
def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

In [44]:
# X = df_train["text"]
# y = df_train["lang_id"]

In [45]:
model_class = BernoulliNB()

In [46]:
params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
         }

In [48]:
model_rfc = GridSearchCV(model_class, params, n_jobs = -1, cv = 10)
# model_rfc = MultinomialNB()

In [49]:
# le = LabelEncoder()
# df_train["lang_id"] = le.fit_transform(df_train["lang_id"])

In [50]:
train, test = feat_CountVec(df_train["text"], df_test["text"])

In [51]:
(X_train, y_train), (X_test, y_test) = train_split(df_train, train)

In [52]:
model_rfc.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 0.5, 1.0, 10.0]})

In [53]:
pred_lr = model_rfc.predict(X_test)

In [54]:
print('Classification Report')
print(classification_report(y_test, pred_lr))

Classification Report
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       583
         eng       1.00      1.00      1.00       615
         nbl       1.00      1.00      1.00       583
         nso       1.00      1.00      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       1.00      1.00      1.00       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       1.00      1.00      1.00       609
         zul       1.00      1.00      1.00       590

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [55]:
cm = confusion_matrix(pred_lr, y_test)
# Printing the accuracy
print("Accuracy of MLPClassifier : ", accuracy(cm))
# 0.9992424242424243

Accuracy of MLPClassifier :  0.9992424242424243


In [56]:
def create_submit(model, X, ind):
    model_submit_results = model.predict(X)
    df_submit = ind
    df_submit["sentiment"] = model_submit_results
    return df_submit

In [57]:
df_submit = create_submit(model_rfc, test, df_test[["index"]])

In [61]:
# def name_kaggle(df, time):
#     now = time
#     df.to_csv(f"Submissions/kwanda_silekwa_{now}.csv", index=False)
def name_kaggle(df, time):
    now = time
    df.to_csv("Kwanda_Silekwa_Classification_Hack.csv", index=False)

In [62]:
creation_date = datetime.now().strftime("%Y%m%d%H%M")

In [63]:
name_kaggle(df_submit, creation_date)

## MultinomialNB

In [64]:
model_class = MultinomialNB()

In [65]:
params = {
    
         }

In [66]:
model_rfc = GridSearchCV(model_class, params, n_jobs = -1, cv = 20)

In [67]:
train, test = feat_CountVec(df_train["text"], df_test["text"])

In [68]:
(X_train, y_train), (X_test, y_test) = train_split(df_train, train)

In [69]:
model_rfc.fit(X_train, y_train)

GridSearchCV(cv=20, estimator=MultinomialNB(), n_jobs=-1, param_grid={})

In [70]:
pred_lr = model_rfc.predict(X_test)

In [71]:
print('Classification Report')
print(classification_report(y_test, pred_lr))

Classification Report
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       583
         eng       1.00      1.00      1.00       615
         nbl       0.99      1.00      1.00       583
         nso       1.00      1.00      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       1.00      1.00      1.00       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       1.00      1.00      1.00       609
         zul       1.00      0.99      1.00       590

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [72]:
cm = confusion_matrix(pred_lr, y_test)
# Printing the accuracy
print("Accuracy of MLPClassifier : ", accuracy(cm))
# 0.9989393939393939

Accuracy of MLPClassifier :  0.9989393939393939


In [73]:
df_submit = create_submit(model_rfc, test, df_test[["index"]])

In [74]:
name_kaggle(df_submit, creation_date)